TODO:
##### Speech cocatenate continuous chunks of audio?

In [ ]:
# !pip install noisereduce

In [1]:
import pyaudio
import numpy as np
from queue import Queue
import threading
import atexit
import sounddevice as sd
import threading
import noisereduce as nr



In [2]:
# Constants for audio stream configuration
FORMAT = pyaudio.paInt16  # Sample format (16-bit)
CHANNELS = 1             # Number of audio channels (1 for mono, 2 for stereo)
# RATE = 44100             # Sample rate (samples per second)
CHUNK_SIZE = 44100        # Number of audio frames per buffer
SAMPLING_RATE = 16000             # Sample rate (samples per second)

DETECT_THRESHOLD = 1400   # Threshold to detect a clap
DETECT_DURATION = 3    # Time in seconds to detect a clap
RECORD_DURATION = 5      # Time in seconds to record audio after clap is detected


In [3]:
# audio_input_queue = Queue(maxsize=20)
audio_input_queue = Queue()
audio_speech2text_queue = Queue()
stop_record_event = threading.Event()


In [4]:


# Define your constants (FORMAT, CHANNELS, RATE, CHUNK_SIZE, RECORD_DURATION, DETECT_THRESHOLD)

def audio_processing():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=SAMPLING_RATE, input=True, frames_per_buffer=CHUNK_SIZE)

    while True:
        data = stream.read(int(DETECT_DURATION * SAMPLING_RATE))
        audio_data_in = np.frombuffer(data, dtype=np.int16)

        # print(audio_data_in[:20])
        # print("detecting:", np.max(audio_data_in))

        if np.max(audio_data_in) < DETECT_THRESHOLD:
            continue
        
        print(np.max(audio_data_in))

        audio_data = audio_data_in
        
        print("audio_data min,max = ", np.min(audio_data), np.max(audio_data), audio_data.shape)

        # Reduce noise from the audio using noisereduce
        audio_noise_reduced = nr.reduce_noise(y=audio_data, sr=SAMPLING_RATE)
        
        # sd.play(audio_noise_reduced, samplerate=SAMPLING_RATE)

        print("audio_noise_reduced min,max = ", np.min(audio_noise_reduced), np.max(audio_noise_reduced), audio_noise_reduced.shape)

        # while np.max(audio_data_in) >= DETECT_THRESHOLD:
        #     data = stream.read(int(RECORD_DURATION * RATE))
        #     audio_data_in = np.frombuffer(data, dtype=np.int16)
        #     print("input voice max = ", np.max(audio_data_in), audio_data_in.shape)
        #     audio_data = np.concatenate((audio_data, audio_data_in))

        # print("audio_data.shape = ", audio_data.shape)
        audio_input_queue.put(audio_noise_reduced)
        

# Create and start the thread as a daemon
audio_input_thread = threading.Thread(target=audio_processing, daemon=True)
audio_input_thread.start()

# You can continue with other tasks here...

# Optionally, you can wait for the audio thread to finish if needed
# audio_input_thread.join()


1821
audio_data min,max =  -1141 1821 (48000,)
audio_noise_reduced min,max =  -1086 1464 (48000,)
3604
audio_data min,max =  -3801 3604 (48000,)
audio_noise_reduced min,max =  -2202 2207 (48000,)
2025
audio_data min,max =  -1153 2025 (48000,)
audio_noise_reduced min,max =  -837 1445 (48000,)
1449
audio_data min,max =  -1168 1449 (48000,)
audio_noise_reduced min,max =  -579 671 (48000,)
1617
audio_data min,max =  -1988 1617 (48000,)
audio_noise_reduced min,max =  -1144 939 (48000,)
2566
audio_data min,max =  -1835 2566 (48000,)
audio_noise_reduced min,max =  -1213 1588 (48000,)
4077
audio_data min,max =  -3726 4077 (48000,)
audio_noise_reduced min,max =  -2531 2821 (48000,)
3613
audio_data min,max =  -2810 3613 (48000,)
audio_noise_reduced min,max =  -2375 2773 (48000,)
2255
audio_data min,max =  -2286 2255 (48000,)
audio_noise_reduced min,max =  -1706 1609 (48000,)
1787
audio_data min,max =  -1831 1787 (48000,)
audio_noise_reduced min,max =  -1714 1689 (48000,)
2692
audio_data min,max 

In [5]:
from pynput import keyboard

def on_press(key):
    if key == keyboard.Key.esc:
        # Stop listener
        return False

def on_release(key):
    pass

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()

In [6]:
while True:
    if not audio_input_queue.empty():
        audio_data = np.zeros((1,))
        # for i in range(10):
        audio_input = audio_input_queue.get()
        audio_input = np.concatenate((audio_input, audio_input_queue.get()))
        sd.play(audio_input, samplerate=SAMPLING_RATE)
        # sd.wait()
        audio_input_queue.task_done()

    if not listener.running:
        break

3858
audio_data min,max =  -2838 3858 (48000,)
audio_noise_reduced min,max =  -2150 2795 (48000,)
1430
audio_data min,max =  -1542 1430 (48000,)
audio_noise_reduced min,max =  -856 358 (48000,)
3871
audio_data min,max =  -3333 3871 (48000,)
audio_noise_reduced min,max =  -1723 2514 (48000,)
8547
audio_data min,max =  -8893 8547 (48000,)
audio_noise_reduced min,max =  -3333 4208 (48000,)
8995
audio_data min,max =  -5611 8995 (48000,)
audio_noise_reduced min,max =  -2784 4027 (48000,)
20757
audio_data min,max =  -12534 20757 (48000,)
audio_noise_reduced min,max =  -7517 10934 (48000,)
9348
audio_data min,max =  -7186 9348 (48000,)
audio_noise_reduced min,max =  -3674 4130 (48000,)
6484
audio_data min,max =  -3356 6484 (48000,)
audio_noise_reduced min,max =  -1902 3804 (48000,)
3474
audio_data min,max =  -2973 3474 (48000,)
audio_noise_reduced min,max =  -1448 1513 (48000,)
7121
audio_data min,max =  -5174 7121 (48000,)
audio_noise_reduced min,max =  -3009 4256 (48000,)
6214
audio_data mi

In [7]:
# def audio_playing():
#     while(True):    
#         audio_data = audio_input_queue.get()
#         # print("C: data = ", data)
#         # data_with_id = f"Task C: {data}"
#         print("receive audio data, len = ", len(audio_data), "type = ", type(audio_data))
#         sd.play(audio_data, SAMPLING_RATE)
#         # sd.wait()
#         audio_input_queue.task_done()

#         if not listener.running:
#             break

# # Create and start the thread as a daemon
# audio_playing_thread = threading.Thread(target=audio_playing, daemon=True)
# audio_playing_thread.start()

In [8]:
# import os
# os.system("pip install git+https://github.com/openai/whisper.git")
# import gradio as gr
import whisper

# model = whisper.load_model("small")
model = whisper.load_model("medium")


1585
1680
1477


In [9]:
def inference(audio):
    if type(audio) == str:
        audio = whisper.load_audio(audio)
    print(type(audio), audio.dtype, audio.shape)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    
    print(result.text)
    return result.text


In [16]:
def audio_speech2text():
    while(True):    
        audio_data = audio_input_queue.get()
        print("stt: received audio data = ", len(audio_data))
        # data_with_id = f"Task C: {data}"
        audio_speech2text = inference(audio_data.astype(np.float32))
        # sd.play(audio_data, SAMPLING_RATE)
        # sd.wait()
        audio_speech2text_queue.put(audio_speech2text)
        audio_input_queue.task_done()

        if not listener.running:
            break

# Create and start the thread as a daemon
audio_speech2text_thread = threading.Thread(target=audio_speech2text, daemon=True)
audio_speech2text_thread.start()

stt: received audio data =  48000
<class 'numpy.ndarray'> float32 (48000,)


1575
1548
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [12]:
def text_printing():
    while(True):    
        text = audio_speech2text_queue.get()
        # print("C: data = ", data)
        # data_with_id = f"Task C: {data}"
        print("receive text data, len = ", len(text), text)
        audio_speech2text_queue.task_done()

        if not listener.running:
            break

# Create and start the thread as a daemon
text_printing_thread = threading.Thread(target=text_printing, daemon=True)
text_printing_thread.start()

4088
4475
3179
1491
1605
2705
1702
2932
4869
2180
1960


In [10]:
import sys
# Function to forcefully terminate all threads
def force_terminate_threads():
    sys.exit()
    audio_input_thread.join()
    audio_playing_thread.join()

# Register the force_terminate_threads function to be called at program exit
atexit.register(force_terminate_threads)

<function __main__.force_terminate_threads()>

4527
4582
8681
6265
3982
7284
3239
1641
